## 1 Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score, balanced_accuracy_score

import warnings
warnings.filterwarnings('ignore')
RANDOM_SEED = 42

## 2 Импорт данных

In [2]:
df = pd.read_csv('speeddating.csv')
print('Размерность датасета: ', df.shape)
df.head()

Размерность датасета:  (8378, 123)


,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,b'',1.0,b'female',21.0,27.0,6.0,b'[4-6]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'0',b'0'
1,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,5.0,b'[6-8]',b'[5-6]',1.0,b'1',b'0',b'0'
2,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'Asian/Pacific Islander/Asian-American',b'1',...,b'[0-3]',b'[3-5]',7.0,NaN,b'[6-8]',b'[0-4]',1.0,b'1',b'1',b'1'
3,b'',1.0,b'female',21.0,23.0,2.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'
4,b'',1.0,b'female',21.0,24.0,3.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'Latino/Hispanic American',b'0',...,b'[0-3]',b'[3-5]',6.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'


## 3 Предварительный анализ данных

In [3]:
df.describe()

,wave,age,age_o,d_age,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,...,music,shopping,yoga,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,like,guess_prob_liked,met
count,8378.000000,8283.000000,8274.000000,8378.000000,8299.000000,8299.000000,8289.000000,8289.000000,8289.000000,8280.000000,...,8299.000000,8299.000000,8299.000000,8220.000000,8277.000000,1800.000000,7205.000000,8138.000000,8069.000000,8003.000000
mean,11.350919,26.358928,26.364999,4.185605,3.784793,3.651645,22.495347,17.396867,20.270759,17.459714,...,7.851066,5.631281,4.339197,0.196010,5.534131,5.570556,3.207814,6.134087,5.207523,0.049856
std,5.995903,3.566763,3.563648,4.596171,2.845708,2.805237,12.569802,7.044003,6.782895,6.085526,...,1.791827,2.608913,2.717612,0.303539,1.734059,4.762569,2.444813,1.841285,2.129565,0.282168
min,1.000000,18.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,-0.830000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,24.000000,24.000000,1.000000,1.000000,1.000000,15.000000,15.000000,17.390000,15.000000,...,7.000000,4.000000,2.000000,-0.020000,5.000000,2.000000,2.000000,5.000000,4.000000,0.000000
50%,11.000000,26.000000,26.000000,3.000000,3.000000,3.000000,20.000000,18.370000,20.000000,18.000000,...,8.000000,6.000000,4.000000,0.210000,6.000000,4.000000,3.000000,6.000000,5.000000,0.000000
75%,15.000000,28.000000,28.000000,5.000000,6.000000,6.000000,25.000000,20.000000,23.810000,20.000000,...,9.000000,8.000000,7.000000,0.430000,7.000000,8.000000,4.000000,7.000000,7.000000,0.000000
max,21.000000,55.000000,55.000000,37.000000,10.000000,10.000000,100.000000,60.000000,50.000000,50.000000,...,10.000000,10.000000,10.000000,0.910000,10.000000,20.000000,18.000000,10.000000,10.000000,8.000000


Эти данные были получены от участников экспериментальных скоростных свиданий в 2002–2004 годах. Во время мероприятий у участников было четырехминутное «первое свидание» со всеми остальными участниками противоположного пола. По истечении четырех минут участников спросили, не хотят ли они снова увидеть свое свидание. Их также попросили оценить свое свидание по шести параметрам: привлекательность, искренность, интеллект, веселье, амбиции и общие интересы. Набор данных также включает данные анкеты, собранные от участников на разных этапах процесса. Эти поля включают в себя: демографию, привычки свиданий, самовосприятие по ключевым характеристикам, убеждения о том, что другие считают ценным в партнере, и информацию об образе жизни.

В датасете имеется 122 столбца(независимые переменные) по которым необходимо предсказать столбец match(зависимая переменная).

In [4]:
df.dtypes.value_counts()

object     64
float64    59
dtype: int64

В 64 столбцах данные имеют формат object(string) и в 59 float64

In [5]:
print((df.isnull().sum() == 0).sum())
print(df.isnull().sum().sum())

66
18173


В 66 столбцах имеется хотя бы одно пропущенное значение, всего пропусков - 18173

In [6]:
df.nunique().sort_values()

has_null                        1
decision                        2
decision_o                      2
samerace                        2
match                           2
                             ... 
shared_interests_important     85
attractive_important           94
pref_o_attractive              94
interests_correlate           155
field                         260
Length: 123, dtype: int64

## 4 Подготовка данных к машинному обучению

Удаляем в столбцы, в которых только одно уникальное значение

In [7]:
drop_column = df.nunique().sort_values().reset_index().rename(columns = {'index':'column'})

In [8]:
drop = [drop_column.column[i] for i in range(len(drop_column)-1) if drop_column[0][i] < 2] 

In [9]:
drop[:5]

['has_null']

In [10]:
df = df.drop(columns=drop, axis=1)
df.head()

,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,1.0,b'female',21.0,27.0,6.0,b'[4-6]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',2.0,...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'0',b'0'
1,1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',2.0,...,b'[0-3]',b'[3-5]',7.0,5.0,b'[6-8]',b'[5-6]',1.0,b'1',b'0',b'0'
2,1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'Asian/Pacific Islander/Asian-American',b'1',2.0,...,b'[0-3]',b'[3-5]',7.0,NaN,b'[6-8]',b'[0-4]',1.0,b'1',b'1',b'1'
3,1.0,b'female',21.0,23.0,2.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',2.0,...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'
4,1.0,b'female',21.0,24.0,3.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'Latino/Hispanic American',b'0',2.0,...,b'[0-3]',b'[3-5]',6.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'


### 4.1 Преобразование текстовых данных

Нужно преобразовать категориальные и текстовые данные в числа

In [11]:
le = LabelEncoder()

In [12]:
def Encoder(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    for feature in columnsToEncode:
      try:
          df[feature] = le.fit_transform(df[feature])
      except:
          print('Error encoding '+feature)
    return df

In [13]:
columnsToEncode = list(df.select_dtypes(include=['category','object']))

In [14]:
for feature in columnsToEncode:
    try:
      df[feature] = le.fit_transform(df[feature])
    except:
      print('Error encoding '+feature)

In [15]:
df = Encoder(df)

In [16]:
df.head()

,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,1.0,0,21.0,27.0,6.0,2,1,3,0,2.0,...,0,1,7.0,6.0,1,1,0.0,1,0,0
1,1.0,0,21.0,22.0,1.0,0,1,3,0,2.0,...,0,1,7.0,5.0,1,1,1.0,1,0,0
2,1.0,0,21.0,22.0,1.0,0,1,1,1,2.0,...,0,1,7.0,NaN,1,0,1.0,1,1,1
3,1.0,0,21.0,23.0,2.0,1,1,3,0,2.0,...,0,1,7.0,6.0,1,1,0.0,1,1,1
4,1.0,0,21.0,24.0,3.0,1,1,4,0,2.0,...,0,1,6.0,6.0,1,1,0.0,1,1,1


### 4.2 Обработка Nan

Пропущенные значения необходимо обработать

In [17]:
df.shape

(8378, 122)

In [18]:
df.dropna().shape

(1048, 122)

Если значения просто удалить, то потеряем больше 80% данных.  
Заполняем nan с помощью метода **knn**

In [19]:
X = df.copy(deep = True)
new_data = df.copy(deep = True)

In [20]:
%%time
imputer=KNNImputer(n_neighbors = 5)
imputer.fit(X)
X1 = imputer.fit_transform(X)

Wall time: 7.9 s


In [21]:
myImputer_data = pd.DataFrame(X1,columns = new_data.columns)

## 5 Построение модели

### 5.1 Разбиваем датасет на тренировочный и тестовый

In [22]:
X = myImputer_data.drop(['match'], axis=1)
y = myImputer_data['match']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED) 

In [24]:
# проверяем
X.shape, X_train.shape, X_test.shape, y.shape, y_train.shape, y_test.shape

((8378, 121), (6283, 121), (2095, 121), (8378,), (6283,), (2095,))

### 5.2 Обучаем модель, генерируем результат

In [25]:
logistic_model = LogisticRegression(random_state=RANDOM_SEED, max_iter=1000)
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [26]:
rfc_model = RandomForestClassifier(random_state=RANDOM_SEED)
rfc_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [27]:
dtc_model = tree.DecisionTreeClassifier(random_state=RANDOM_SEED)
dtc_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

## 6 Оценка качества модели

### 6.1 Создаем таблицу с метриками для логистической регрессии, случайного леса и дерева решений

In [28]:
def create_metrics(model, X, y):
    lst = []
    lst.append(accuracy_score(model.predict(X), y))
    lst.append(balanced_accuracy_score(model.predict(X), y))
    lst.append(precision_score(model.predict(X), y))
    lst.append(recall_score(model.predict(X), y))
    lst.append(f1_score(model.predict(X), y))
    lst.append(roc_auc_score(model.predict(X), y))
    return lst

In [29]:
metrics = ['accuracy', 'balanced accuracy', 'precision', 
           'recall', 'f1_score', 'roc_auc']


df_metrics = pd.DataFrame({'metrics': metrics, 
                           'pred_train_logistic': create_metrics(logistic_model, X_train, y_train), 
                           'pred_test_logistic': create_metrics(logistic_model, X_test, y_test),
                           'pred_train_RandomForest': create_metrics(rfc_model, X_train, y_train), 
                           'pred_test_RandomForest': create_metrics(rfc_model, X_test, y_test),
                           'pred_train_DecisionTree': create_metrics(dtc_model, X_train, y_train), 
                           'pred_test_DecisionTree': create_metrics(dtc_model, X_test, y_test)
                          }
                         )
df_metrics.set_index('metrics')

,pred_train_logistic,pred_test_logistic,pred_train_RandomForest,pred_test_RandomForest,pred_train_DecisionTree,pred_test_DecisionTree
metrics,,,,,,
accuracy,1.0,1.0,1.0,0.995227,1.0,1.0
balanced accuracy,1.0,1.0,1.0,0.997143,1.0,1.0
precision,1.0,1.0,1.0,0.971831,1.0,1.0
recall,1.0,1.0,1.0,1.000000,1.0,1.0
f1_score,1.0,1.0,1.0,0.985714,1.0,1.0
roc_auc,1.0,1.0,1.0,0.997143,1.0,1.0
